## In this notebook we will try to train a simple pytorch model on the Iris dataset

In [60]:
import torch
import sklearn
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch.nn as nn
import torch.nn.functional as F

In [61]:

#loading iris dataset and extracting variables
iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

#no of targets
print(names)

['setosa' 'versicolor' 'virginica']


## Model

In [62]:
class Model(nn.Module):
    
    #defining model structure
    def __init__(self,input_dims):
        super(Model,self).__init__()
        self.layer1 = nn.Linear(input_dims,100)
        self.layer2 = nn.Linear(100,50)
        self.layer3 = nn.Linear(50,3)
        
    #define forward computation with activation functions
    def forward(self,x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x))
        
        return x
    

In [63]:
#instantiating our model and defining optimizer and loss fn
model = Model(4)
optimizer = torch.optim.Adagrad(model.parameters(),lr = 0.01)
loss_fn = nn.CrossEntropyLoss()
print(model)

Model(
  (layer1): Linear(in_features=4, out_features=100, bias=True)
  (layer2): Linear(in_features=100, out_features=50, bias=True)
  (layer3): Linear(in_features=50, out_features=3, bias=True)
)


## Preparing our data

In [64]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

X_scaled = X_scaled.astype(np.float32)

X_train,X_test,y_train,y_test = train_test_split(X_scaled,y,test_size=0.2,random_state=2)


In [65]:
# we need to convert out dataset variables to tensors to utilize gpu computation

X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)




## Training

In [67]:
EPOCHS = 100

for epoch in range(EPOCHS):
    
    y_pred = model(X_train)
    loss = loss_fn(y_pred,y_train)
    
    if epoch % 10 == 0:
        print ("Epoch #",epoch)
        print(loss.item())
        
    #Zero the gradients
    optimizer.zero_grad()
    loss.backward()#calculate gradients
    optimizer.step()#update weights
    

Epoch # 0
1.0889053344726562
Epoch # 10
0.709537923336029
Epoch # 20
0.6378505229949951
Epoch # 30
0.6061201691627502
Epoch # 40
0.5926541090011597
Epoch # 50
0.5853510499000549
Epoch # 60
0.5807108283042908
Epoch # 70
0.5774818062782288
Epoch # 80
0.5751286745071411
Epoch # 90
0.573313295841217


/tmp/ipykernel_12515/2796111341.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.layer3(x))


## Predictions

In [68]:
with torch.no_grad():
    y_test_pred = model(X_test)

/tmp/ipykernel_12515/2796111341.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.layer3(x))


In [74]:
accuracy_score(np.argmax(y_test_pred,axis=1),y_test)

0.9333333333333333